In [1]:
""" 1. Export your Postman Collection, then make an instance of the Postman runner. """
from pyclinic.postman import Postman
from rich import print

collection_path = "./tests/examples/deckofcards.postman_collection.json"
runner = Postman(collection_path)

http://deckofcardsapi.com/api/deck/{{deck_id}}/shuffle/ => Variable deck_id not found in collection, environment, or global variables.
http://deckofcardsapi.com/api/deck/{{deck_id}}/draw/?count=2 => Variable deck_id not found in collection, environment, or global variables.
http://deckofcardsapi.com/api/deck/{{deck_id}}/draw/?count=2 => Variable deck_id not found in collection, environment, or global variables.
http://deckofcardsapi.com/api/deck/{{deck_id}}/shuffle/ => Variable deck_id not found in collection, environment, or global variables.
http://deckofcardsapi.com/api/deck/{{deck_id}}/pile/{{pile_name}}/list/ => Variable deck_id not found in collection, environment, or global variables.
http://deckofcardsapi.com/api/deck/{{deck_id}}/pile/{{pile_name}}/list/ => Variable pile_name not found in collection, environment, or global variables.
http://deckofcardsapi.com/api/deck/{{deck_id}}/pile/{{pile_name}}/list/ => Variable deck_id not found in collection, environment, or global variab

In [2]:
""" 2. Did you see the warnings that were printed above?

That's because there are variables, like {{deck_id}}, that are not defined in the collection!
So calling the functions as-is will fail... You should consider when, where and how you use variables in Postman to solve this.

But, let's see which variables we _do_ have defined. 
"""
runner.show_variables()

────────────────────── Deck of Cards instantiated with these Variables ──────────────────────

{}

In [3]:
""" 3. An empty dictionary means that there are zero variables defined...

Let's see what folders and functions we have.
"""
runner.show_folders()

───────────────────── Folders found in Deck of Cards Postman Collection ─────────────────────

*Root.

        create_shuffled_deck

        draw_cards

        list_cards_in_piles

...

        draw_cards

Folder2.

        list_cards_in_piles

Example Usage

response = runner.Folder2.list_cards_in_piles()

In [4]:
""" 4. We have a lot and they're ready to be used!

A few things to note:
    - The Root folder has all of the functions available since it's the top-level
        * However, notice how create_shuffled_deck is only defined at the Root, just like in the Postman Collection
    
    - In Postman, Folder11 was actually called Folder 1.1 and was a subfolder of Folder 1
        * To make things easier to work with, we flatten all the folders and normalize the names

    - Each other folder shows which functions belong to it
        * This makes it easy to execute doing something like runner.Folder2.list_cards_in_piles()
        * Just like it shows in the Example Usage
    
    - You'll get errors if you try to use a function that doesn't exist in a folder
        * We're about to see an error because we're missing the {{deck_id}} variable...
"""
# runner.Folder11.list_cards_in_piles() # This will fail because list_cards_in_piles is not defined in Folder11
response = runner.Folder11.draw_cards()
print("STATUS CODE:", response.status_code)
print(response.text)

STATUS CODE: 404

<!doctype html>
<html lang="en">
<head>
  <title>Not Found</title>
</head>
<body>
  <h1>Not Found</h1><p>The requested resource was not found on this server.</p>
</body>
</html>

In [5]:
""" 5. Notice how we're working with a response. This comes from the requests library.

Using type hinting, we will get more intellisense. Now we can work with the response a lot more.

Let's use an auto-generated function that we know will work without needing variables.
"""
from requests import Response

response: Response = runner.Root.create_shuffled_deck()
print(response.json())

{'success': True, 'deck_id': '74il47d7ba3r', 'remaining': 52, 'shuffled': True}

In [6]:
""" 6. AMAZING! Let's try drawing cards again, but this time we'll pass in a {{deck_id}} so it works. """
create_response = runner.Root.create_shuffled_deck()
deck_id = create_response.json().get("deck_id")

response = runner.Folder11.draw_cards({"deck_id": deck_id})
print("STATUS CODE:", response.status_code)
print(response.json())

STATUS CODE: 200

{
    'success': True,
    'deck_id': '5ec6p0z6z0ad',
    'cards': [
        {
            'code': '9H',
            'image': 'https://deckofcardsapi.com/static/img/9H.png',
            'images': {
                'svg': 'https://deckofcardsapi.com/static/img/9H.svg',
                'png': 'https://deckofcardsapi.com/static/img/9H.png'
            },
            'value': '9',
            'suit': 'HEARTS'
        },
        {
            'code': 'KD',
            'image': 'https://deckofcardsapi.com/static/img/KD.png',
            'images': {
                'svg': 'https://deckofcardsapi.com/static/img/KD.svg',
                'png': 'https://deckofcardsapi.com/static/img/KD.png'
            },
            'value': 'KING',
            'suit': 'DIAMONDS'
        }
    ],
    'remaining': 50
}

In [7]:
""" 7. SUCCESS! If you wanted this to be an Automated Test with pytest, it would look something like this: """
def test_draw_cards():
    runner = Postman(collection_path)
    create_response = runner.Root.create_shuffled_deck()
    deck_id = create_response.json().get("deck_id")

    # Pass in User Variables with a flat dictionary
    response = runner.Folder11.draw_cards({"deck_id": deck_id})
    body = response.json()

    assert response.ok
    assert body["success"] is True
    assert len(body["cards"]) == 2, "By default, two cards should be drawn"

In [8]:
""" 8. Using your Environment and Global Variables.

If your Variables are at the Environment and/or Global scope in Postman,
then export them as files and pass in the file paths when instantiating the Postman runner.

* In this cell's output, observe that there are 4 functions that use the {{USER_ID}} variable,
    yet the value is None or empty.
* We then see the Variables Dictionary and, sure enough, there are a lot of empty value fields.
"""
from tests import utils

GLOBAL_PATH = utils.WORKSPACE_GLOBAL_VARIABLES_PATH
ENV_PATH = utils.BOOKSTORE_ENV_VARIABLES_PATH
COLLECTION_PATH = utils.build_example_path(utils.BOOKSTORE_PATH)

user_variables = {"USERNAME": "Carlos Kidman"}
runner = Postman(COLLECTION_PATH, ENV_PATH, GLOBAL_PATH, user_variables)
runner.show_variables()

https://demoqa.com/Account/v1/User/{{USER_ID}} => Variable USER_ID was found, but value was set to None or empty.
https://demoqa.com/Account/v1/User/{{USER_ID}} => Variable USER_ID was found, but value was set to None or empty.
https://demoqa.com/Account/v1/User/{{USER_ID}} => Variable USER_ID was found, but value was set to None or empty.
https://demoqa.com/Account/v1/User/{{USER_ID}} => Variable USER_ID was found, but value was set to None or empty.


───────────────────── Book Store API instantiated with these Variables ──────────────────────

{
    'NAME': {'value': 'CARLOS KIDMAN', 'enabled': True},
    'BASE_URL': {'value': 'https://demoqa.com', 'enabled': True},
    'USER_ID': {'value': '', 'enabled': True},
    'USERNAME': {'value': 'Carlos Kidman', 'enabled': True},
    'PASSWORD': {'value': '', 'enabled': True},
    'TOKEN': {'value': '', 'enabled': True}
}

# Postman Variables

It's important to understand when, where and how to use Postman's Variables. It'll make your automation much easier to work with.

## Order of Operations

When working with PyClinic, there are 4 levels of scopes that will override each other if there are matching Variables:

1. Global
2. Environment
3. Collection
4. User

In the above order, each scope's Variables are loaded and override any matching keys that exist.

For example, if Global and Environment both have a USERNAME Variable, then the Environment Scope's value will win.
That means that any User Variables will ultimately win and have their values be set.

* NOTE: Only matching values are overriden. Otherwise, the Variables dictionary is just updated with the new key-value pairs.

In [9]:
""" 9. Create your own service functions.

Although PyClinic aims to bootstrap your Project for automation and test automation,
you will most likely need to create your own service functions for more control and power.

The requests library that PyClinic uses is wonderful for this!
You should already have this info in Postman, and help from API docs and your team,
but you can also get help from PyClinic.
"""
# What if we wanted to change the number of decks that are created (maybe to 3 or n)?
# We can use the .help() method on the function to get more info.

runner = Postman(collection_path)
runner.Root.create_shuffled_deck.help()

http://deckofcardsapi.com/api/deck/{{deck_id}}/shuffle/ => Variable deck_id not found in collection, environment, or global variables.
http://deckofcardsapi.com/api/deck/{{deck_id}}/draw/?count=2 => Variable deck_id not found in collection, environment, or global variables.
http://deckofcardsapi.com/api/deck/{{deck_id}}/draw/?count=2 => Variable deck_id not found in collection, environment, or global variables.
http://deckofcardsapi.com/api/deck/{{deck_id}}/shuffle/ => Variable deck_id not found in collection, environment, or global variables.
http://deckofcardsapi.com/api/deck/{{deck_id}}/pile/{{pile_name}}/list/ => Variable deck_id not found in collection, environment, or global variables.
http://deckofcardsapi.com/api/deck/{{deck_id}}/pile/{{pile_name}}/list/ => Variable pile_name not found in collection, environment, or global variables.
http://deckofcardsapi.com/api/deck/{{deck_id}}/pile/{{pile_name}}/list/ => Variable deck_id not found in collection, environment, or global variab

──────────────────────── Request Dictionary for create_shuffled_deck ────────────────────────

{
    'method': 'GET',
    'url': 'http://deckofcardsapi.com/api/deck/new/shuffle/?deck_count=1',
    'headers': {}
}

In [10]:
""" 10. Write a function using the info we have.

Looking at the info above, there's really only two pieces we need:
    - The URL
    - The HTTP Method
"""
import requests

def create_decks(count=1):
    method = "GET"
    url = f"http://deckofcardsapi.com/api/deck/new/shuffle/?deck_count={count}"
    response = requests.request(method, url)
    return response.json()

# Instead of 52 cards for a single deck, we should now have 3 decks with 156 cards total
print(create_decks(3))

{'success': True, 'deck_id': 'b5lge31fdotb', 'remaining': 156, 'shuffled': True}